<a href="https://colab.research.google.com/github/josephmailil1/Speciale---Stibo/blob/main/Agent%20Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
from google.colab import userdata
openai_api_key = userdata.get('OpenAI_Key') #use the name of your colab key

In [43]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [44]:
import os
os.chdir("/content/drive/My Drive/Speciale")

In [47]:
import pandas as pd
shap_df = pd.read_csv("shap_noincome100.csv")
data_df = pd.read_csv("basedata100.csv")

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

tool = PythonAstREPLTool(locals={"data": data_df, "shap": shap_df})
tool.invoke("data['age'].mean()")

37.5

In [13]:
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)

In [14]:
from langchain.output_parsers import JsonOutputToolsParser

In [15]:
parser = JsonOutputToolsParser(tool = tool)

In [18]:
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

system = f"""You have access to a pandas dataframe 'data'. This is the dataframe containing actual observations and values of the dataset \
Here is the output of 'data.head().to_markdown()':

```
{data_df.head().to_markdown()}
```
You are also given the pandas dateframe 'shap'. Containing shap values for the test set obsevations. \
The shap values shows how each feature impacts the y-variable income on a local prediction level. The actual Y-variable is not available in this dataset. \
When asked to explain something, use the shap values for the observations to substantiate your answer.
```

{shap_df.head().to_markdown()}
```

The dataset contains 13K observations of descriptive data on individuals, with the prediction task being to determine whether a person makes over $50K a year.

Given a user question, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Follow these steps: \
  1. Generete the python code you want to use. \
  2. Execute The Python Code \
  3. Answer the Question based on the exececuted Python Code in Step 2 \
  4. Return both the answer and the Python Code you execeuted marked in []."""


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system,
        ),
        ("human", "{question}"),
        # This MessagesPlaceholder allows us to optionally append an arbitrary number of messages
        # at the end of the prompt using the 'chat_history' arg.
        MessagesPlaceholder("chat_history", optional=True),
    ]
)


def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]


chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tools)
    .assign(tool_output=itemgetter("ai_msg") | parser )
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["response"])
)

In [19]:
from langchain.memory import ConversationBufferMemory
# This is do give the bot memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [20]:
def call_chain(user_question):
    answer = chain.invoke({"question": user_question})
    return answer['response']

# Interface - Gradio

In [ ]:
pip install gradio

In [ ]:
import gradio as gr

In [ ]:
demo = gr.Interface(
    fn=call_chain,
    inputs=["text"],
    outputs=["text"]
)

In [ ]:
demo.launch()

In [ ]:
shap.loc[0, ['marital-status_Married-civ-spouse', 'education-num', 'occupation_Other-service', 'sex_Male']]

In [ ]:
shap.loc[0].nlargest(5)

In [ ]:
important_features = shap.iloc[0].sort_values(ascending=False)
print(important_features)

In [ ]:
shap['marital-status_Married-civ-spouse'].mean()

In [21]:
!pip install --quiet python-dotenv
!pip install --quiet openai
!pip install --quiet docarray
!pip install --quiet tiktoken #  This is needed in order to for OpenAIEmbeddings
!pip install --quiet cohere
pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.6 MB/s eta 0:00:00


In [24]:
from langchain.chains import RetrievalQA # will do retrival over documents
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader # used to load propriatory data
from langchain.document_loaders import DirectoryLoader  # used to load propriatory data
from langchain.vectorstores import DocArrayInMemorySearch # load an in-memory vector store
from IPython.display import display, Markdown # common utilities for displaying information in notebooks.

In [78]:
path = os.getcwd()

In [48]:
loader = DirectoryLoader(path = path, glob="**/*.csv", use_multithreading = True, loader_cls = CSVLoader)

In [49]:
data = loader.load()

In [54]:
print(data)

[Document(page_content='age: 30\nfnlwgt: 264351\neducation-num: 13\ncapital-gain: 0\ncapital-loss: 0\nhours-per-week: 25\nworkclass_Local-gov: False\nworkclass_Private: True\nworkclass_Self-emp-inc: False\nworkclass_Self-emp-not-inc: False\nworkclass_State-gov: False\nworkclass_Without-pay: False\neducation_11th: False\neducation_12th: False\neducation_1st-4th: False\neducation_5th-6th: False\neducation_7th-8th: False\neducation_9th: False\neducation_Assoc-acdm: False\neducation_Assoc-voc: False\neducation_Bachelors: True\neducation_Doctorate: False\neducation_HS-grad: False\neducation_Masters: False\neducation_Preschool: False\neducation_Prof-school: False\neducation_Some-college: False\nmarital-status_Married-AF-spouse: False\nmarital-status_Married-civ-spouse: True\nmarital-status_Married-spouse-absent: False\nmarital-status_Never-married: False\nmarital-status_Separated: False\nmarital-status_Widowed: False\noccupation_Armed-Forces: False\noccupation_Craft-repair: False\noccupation

In [80]:
from langchain.indexes import VectorstoreIndexCreator

In [81]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key)

In [53]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding = embeddings
).from_loaders([loader])

In [56]:
db = DocArrayInMemorySearch.from_documents(
    data,
    embeddings
)

In [57]:
retriever = db.as_retriever()

In [58]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [75]:
query =  "Please give me the names of the 4 data set you have access to?"

In [76]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [77]:
display(Markdown(response))

I don't have access to the names of the datasets.

In [ ]:
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

system = f"""You have access to a pandas dataframe 'data'. This is the dataframe containing actual observations and values of the dataset \
Here is the output of 'data.head().to_markdown()':

```
{data.head().to_markdown()}
```
You are also given the pandas dateframe 'shap'. Containing shap values for the test set obsevations. \
The shap values shows how each feature impacts the y-variable income on a local prediction level. The actual Y-variable is not available in this dataset. \
When asked to explain something, use the shap values for the observations to substantiate your answer.
```

{shap.head().to_markdown()}
```

The dataset contains 13K observations of descriptive data on individuals, with the prediction task being to determine whether a person makes over $50K a year.

Given a user question, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Follow these steps: \
  1. Generete the python code you want to use. \
  2. Execute The Python Code \
  3. Answer the Question based on the exececuted Python Code in Step 2 \
  4. Return both the answer and the Python Code you execeuted marked in []."""


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system,
        ),
        ("human", "{question}"),
        # This MessagesPlaceholder allows us to optionally append an arbitrary number of messages
        # at the end of the prompt using the 'chat_history' arg.
        MessagesPlaceholder("chat_history", optional=True),
    ]
)


def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]


chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tools)
    .assign(tool_output=itemgetter("ai_msg") | parser )
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["response"])
)